In [ ]:
!pip install eo-learn -q

     |████████████████████████████████| 10.5 MB 7.3 MB/s 
     |████████████████████████████████| 69 kB 9.5 MB/s 
     |████████████████████████████████| 131 kB 64.2 MB/s 
     |████████████████████████████████| 201 kB 65.6 MB/s 
     |████████████████████████████████| 1.0 MB 48.2 MB/s 
     |████████████████████████████████| 133 kB 66.6 MB/s 
     |████████████████████████████████| 53.8 MB 255 kB/s 
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
     |████████████████████████████████| 994 kB 40.0 MB/s 
     |████████████████████████████████| 15.4 MB 40.8 MB/s 
     |████████████████████████████████| 6.3 MB 46.1 MB/s 
     |████████████████████████████████| 3.1 MB 34.6 MB/s 
     |████████████████████████████████| 8.4 MB 27.7 MB/s 
     |████████████████████████████████| 128 kB 75.7 MB/s 
     |████████████████████████████████| 79 kB 9.9 MB/s 
     |████████████████████████████████| 127 kB 68.4 MB/s 
     |████████████████████████████████| 47 kB 5.5 MB/s 
     |█████████████

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, glob
import cv2
import eolearn
import gdown
from pathlib import Path
import zipfile

from eolearn.core import EOPatch, FeatureType

In [ ]:
TARGET_SIZE = (64, 64)

## Новый раздел

In [ ]:
def read_file(filepath):
    eopatch = EOPatch.load(filepath)
    img = eopatch.data['L2A']
    masks = [eopatch.mask[x] for x in eopatch.mask]

    return np.concatenate([img] + masks, axis=-1)

## CCT

### Гиперпараметры

In [ ]:
positional_emb = True
conv_layers = 2
projection_dim = 128

num_heads = 2
transformer_units = [
    projection_dim,
    projection_dim,
]
transformer_layers = 2
stochastic_depth_rate = 0.1

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 30
image_size = 64
input_shape = [64, 64, 19]

### Токенайзер

In [ ]:

class CCTTokenizer(layers.Layer):
    def __init__(
        self,
        kernel_size=3,
        stride=1,
        padding=1,
        pooling_kernel_size=3,
        pooling_stride=2,
        num_conv_layers=conv_layers,
        num_output_channels=[64, 128],
        positional_emb=positional_emb,
        **kwargs,
    ):
        super(CCTTokenizer, self).__init__(**kwargs)

        self.conv_model = keras.Sequential()
        for i in range(num_conv_layers):
            self.conv_model.add(
                layers.Conv2D(
                    num_output_channels[i],
                    kernel_size,
                    stride,
                    padding="valid",
                    use_bias=False,
                    activation="relu",
                    kernel_initializer="he_normal",
                )
            )
            self.conv_model.add(layers.ZeroPadding2D(padding))
            self.conv_model.add(
                layers.MaxPool2D(pooling_kernel_size, pooling_stride, "same")
            )

        self.positional_emb = positional_emb

    def call(self, images):
        outputs = self.conv_model(images)
        # После прохождения данной мини-сетки изображение превращается в ряд последовательностей
        reshaped = tf.reshape(
            outputs,
            (-1, tf.shape(outputs)[1] * tf.shape(outputs)[2], tf.shape(outputs)[-1]),
        )
        return reshaped

    def positional_embedding(self, image_size):
        # Опциональная часть с позиционным эмбеддингом
        if self.positional_emb:
            dummy_inputs = tf.ones((1, image_size, image_size, 3))
            dummy_outputs = self.call(dummy_inputs)
            sequence_length = tf.shape(dummy_outputs)[1]
            projection_dim = tf.shape(dummy_outputs)[-1]

            embed_layer = layers.Embedding(
                input_dim=sequence_length, output_dim=projection_dim
            )
            return embed_layer, sequence_length
        else:
            return None


### Stochastic depth

[Stochastic depth](https://arxiv.org/abs/1603.09382) -
техника регуляризации, которая случайным образом блокирует работу некоторых слоев нейросети. По смыслу близка к "дропауту" - [Dropout](https://jmlr.org/papers/v15/srivastava14a.html), с той разницей, что дропаут блокирует отдельные нейроны.

In [ ]:
# Источник: github.com:rwightman/pytorch-image-models.
class StochasticDepth(layers.Layer):
    def __init__(self, drop_prop, **kwargs):
        super(StochasticDepth, self).__init__(**kwargs)
        self.drop_prob = drop_prop

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_prob
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x


### multilayer perceptron (MLP) для энкодера трансформера

In [ ]:

def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


### Финальная сборка модели

In [ ]:

def create_cct_model(
    input_shape=input_shape,
    image_size=image_size,
    num_heads=num_heads,
    projection_dim=projection_dim,
    transformer_units=transformer_units,
):

    inputs = layers.Input(input_shape)

    # Кодирование (нарезка) патчей.
    cct_tokenizer = CCTTokenizer()
    encoded_patches = cct_tokenizer(inputs)

    # Применение позиционного эмбеддинга.
    
    # Stochastic Depth
    dpr = [x for x in np.linspace(0, stochastic_depth_rate, transformer_layers)]

    # Блоки трансформера.
    for i in range(transformer_layers):
        # Нормализация.
        x1 = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)

        # Self-Attention-блок.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # Skip-connection.
        attention_output = StochasticDepth(dpr[i])(attention_output)
        x2 = layers.Add()([attention_output, encoded_patches])

        # Нормализация.
        x3 = layers.LayerNormalization(epsilon=1e-5)(x2)

        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)

        # Skip-connection.
        x3 = StochasticDepth(dpr[i])(x3)
        encoded_patches = layers.Add()([x3, x2])

    # Применение sequence pooling для получения взвешенного выхода сети.
    representation = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
    attention_weights = tf.nn.softmax(layers.Dense(1,)(representation), axis=1)
    weighted_representation = tf.matmul(
        attention_weights, representation, transpose_a=True
    )
    weighted_representation = tf.squeeze(weighted_representation, -2)

    # Выход классификатора.
    logits = layers.Dense(1,  activation='sigmoid')(weighted_representation)
    # Готовая модель
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


# Новый раздел

In [ ]:
model = create_cct_model(input_shape = [64, 64, 19])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.Accuracy(name="accuracy")
    ],
)


In [ ]:
model.load_weights('CCT.h5')

## Новый раздел

In [ ]:
!unzip -q /content/CCT-inference.zip -d /content/example/

In [ ]:
filepath = input('input path to folder: ')

input_data = read_file(filepath)

model(input_data, training=False)

input path to folder: /content/example/content/spills/2021-01/КБ-2020-1


<tf.Tensor: shape=(28, 1), dtype=float32, numpy=
array([[0.00053731],
       [0.01329342],
       [0.01402196],
       [0.01625428],
       [0.01519221],
       [0.00520554],
       [0.03364789],
       [0.026039  ],
       [0.03156918],
       [0.01835027],
       [0.0322507 ],
       [0.03923708],
       [0.02190122],
       [0.03197056],
       [0.01881793],
       [0.02813515],
       [0.01391655],
       [0.03489986],
       [0.03116351],
       [0.02107072],
       [0.00860626],
       [0.00856319],
       [0.0082854 ],
       [0.01715851],
       [0.0142059 ],
       [0.00862572],
       [0.0087719 ],
       [0.00953099]], dtype=float32)>

In [ ]:
input_data.shape

(28, 64, 64, 19)